In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.part'):
    shutil.rmtree('./file/3.part')

os.makedirs('./file/3.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.url/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(input_.loc[a, 'Url'],
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                market = html.xpath('//div[@class="part__canada-ligne"]/img/@title')[0].strip() if html.xpath('//div[@class="part__canada-ligne"]/img') else ''

                # = = = = = = = = = = = = = = =

                type_ = ''.join([text.strip() for text in html.xpath('//div[@class="part__canada-ligne"]/text()')]).strip()
                
                # = = = = = = = = = = = = = = =

                part_number = html.xpath('//h1[@class="part__titre-code"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =

                list_div = html.xpath('//dl[@class="part__specifications"]/div[@class="part__nom-spec-valeur-spec"]')
                
                # = = = = = = = = = = = = = = =

                dict_specification = {}
                for i, div in zip(range(len(list_div)), list_div):
                    dict_specification[str(i)] = {div.xpath('./dt[@class="part__nom-spec"]/text()')[0].strip(): div.xpath('./dd[@class="part__valeur-spec"]/text()')[0].strip()}

                # = = = = = = = = = = = = = = =

                list_src = [f'https://ecat.spectrapremium.com{src.strip()}' for src in html.xpath('//a[@class="part__magic-scroll-images__thumbnail"]/@href')]
                while 'https://ecat.spectrapremium.com#' in list_src:
                    list_src.remove('https://ecat.spectrapremium.com#')
    
                # = = = = = = = = = = = = = = =

                dict_src = {}
                for i, src in zip(range(len(list_src)), list_src):
                    dict_src[str(i)] = src
    
                # = = = = = = = = = = = = = = =

                vehicle_page = int(html.xpath('//li[@class="paginator__last"]/a/@href')[0].strip().removesuffix('#applications').split('=')[-1]) if html.xpath('//li[@class="paginator__last"]') else 1

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Market': market,
                                         'Type': type_,
                                         'Part Number': part_number,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Specification': json.dumps(dict_specification),
                                         'Vehicle Page': vehicle_page}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(10):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.part/{file}',
              f'''./file/3.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[状态：ok，尝试次数：2] - 100.00% - crawler_9 > 325.https://ecat.spectrapremium.com/en/parts/0668452
[剩余数量：0] - [当前时间：12:23:52]

输出ing...



Progress: 100%|████████████████████████████| 43/43 [00:00<00:00, 7594.54it/s]

Done ~
